In [1]:
# import cv2 아직 확정은 아님
# import tkinter 아직 확정은 아님
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier


#에이 진짜 설마 보고 그러겠어 에이설마 진짜로ㅡㅡ

def  survey(): # 팀원추가 def 2가지로 나누기
  jj_list = ['a','b'] # a,b는 문제임
  for i,j in enumerate(jj_list):

    """
    enumerate를 하는 이유
    몇번 부터 몇번까지 해당 항목에 가중치를 추가해야하기 떄문에
    """
    print(j)
    aa = int(input("1,2,3,4,5 중 하나 선택"))

  """
  이후 질문당 가중치 설정해 각 변수의 값구하기
  """


def csv_to(x):
  """
  2차원 리스트를 받아 [[]] 판다스를 통해 csv로 변환
  """


def my_profile():
  my_profile = np.array([4, 4, 4, 3]) # 설문조사이후 이와 같이 저장하도록함 (1차원배열) 굳이 넘파이 X


def predic(load):
  """
  매번 실행마다 학습하는것을 방지하기 위해 경로입력후 학습 한번만 시키기
  """


  global knn, data_y

  data = pd.read_csv(rf'{load}') # 경로

  data_x = data.iloc[:, :-1]
  data_y = data.iloc[:, -1]

  knn = KNeighborsClassifier(n_neighbors=len(data))

  knn.fit(data_x, data_y)


def model(profile): # 내 정보 입력
  """
  k-nn 모델 학습 학습후
  팀 인원수 n입력하여 가장 잘맞는 팀원 n 구하기
  """

  if knn is None or data_y is None:
    print('학습을 먼저하고 와주세요.')
    return

  n = int(input('원하는 팀원의 수 입력 : '))

  distances, peoples = knn.kneighbors([profile], n_neighbors=n, return_distance=True)

  for most, label in enumerate(peoples):
    print(f"{most+1}번째로 가까운 사람 : {data_y.iloc[label]}")


def main():
  """
  GUI
  """

if __name__ == '__main__':
  main()


1


K-nn예제

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# 예시 설문조사 데이터 (팀원들의 성향을 수치화한 데이터)
data = np.array([
    [4, 5, 3, 2],  # 팀원 1
    [5, 5, 4, 3],  # 팀원 2
    [2, 1, 5, 4],  # 팀원 3
    [3, 4, 3, 2],  # 팀원 4
    [5, 5, 5, 5],  # 팀원 5
])

# 나의 설문조사 응답
my_profile = np.array([4, 4, 4, 3])

# 팀원 이름 라벨
labels = np.array(["Alice", "Bob", "Charlie", "David", "Eve"])

# KNN 모델 (k=3)
knn = KNeighborsClassifier(n_neighbors=len(data))

# 모델 학습
knn.fit(data, labels)

# 나와 가장 가까운 팀원 3명을 찾고, 거리와 이웃 정보 반환
distances, indices = knn.kneighbors([my_profile], return_distance=True)

# 가장 가까운 팀원의 이름을 출력
print(indices)
nearest_team_members = labels[indices]
print("가장 가까운 팀원들의 이름:", nearest_team_members)
print("이웃들과의 거리:", distances)
